Moving on to the next step, the last columns containg cordinates and population are included in the dataframe for each year before merging them into a single dataframe. 

In [1]:
# Importing the libraries
import pandas as pd
import time
from geopy.geocoders import Nominatim

In [ ]:
# Importing the dataset for 2016 with skills. The whole process is repeated all years.
df_16_skills = pd.read_csv('updated_job_listings_with_skills_2016.csv')

In [ ]:
# Initialize the geocoder
geolocator = Nominatim(user_agent="geoapi")

# Cache to store already processed locations
location_cache = {}

# Function to fetch latitude and longitude with caching and delay
def fetch_coordinates(area_name):
    if area_name in location_cache:
        return location_cache[area_name]
    try:
        location = geolocator.geocode(area_name)
        time.sleep(1)  # Delay to avoid rate-limiting
        if location:
            coords = (location.latitude, location.longitude)
        else:
            coords = (None, None)
        location_cache[area_name] = coords
        return coords
    except Exception as e:
        return (None, None)

# Apply the function to the 'Area' column in the DataFrame
df_16_skills['Coordinates'] =df_16_skills['Area'].apply(fetch_coordinates)
df_16_skills['Latitude'] = df_16_skills['Coordinates'].apply(lambda x: x[0])
df_16_skills['Longitude'] = df_16_skills['Coordinates'].apply(lambda x: x[1])

# Drop the intermediate 'Coordinates' column
df_16_skills = df_16_skills.drop(columns=['Coordinates'])

# Looking at the first few rows of the dataset
df_16_skills

In [ ]:
# Mapping of Enligsh to Danish city names as the API does not recognize English names

city_name_mapping = {
    'Copenhagen': 'København',
    'Aarhus': 'Århus',
    'Elsinore': 'Helsingør'
}

df_16_skills['Area'] = df_16_skills['Area'].replace(city_name_mapping)

# call Århus for Aarhus
df_16_skills['Area'] = df_16_skills['Area'].replace('Århus', 'Aarhus')


import pandas as pd
import requests
from io import StringIO

# Define the parameters for the population data request
params = {
    'table': 'FOLK1A',  # Table for population
    'format': 'BULK',
    'variables': [
        {'code': 'OMRÅDE', 'values': ['*']},  # All areas
        {'code': 'Tid', 'values': [f'{year}K1' for year in range(2016, 2025)]},  # First quarter of each year from 2016 to 2024
        {'code': 'KØN', 'values': ['TOT']},  # Total population (ignore gender)
        {'code': 'ALDER', 'values': ['IALT']},  # All ages
        {'code': 'CIVILSTAND', 'values': ['TOT']}  # Total population (ignore marital status)
    ]
}

# Make the request to the API
response = requests.post('https://api.statbank.dk/v1/data', json=params)
data = response.text

# Check for errors in the response
if 'errorTypeCode' in data:
    print("Error:", data)
else:
    # Load the response into a pandas DataFrame
    population = pd.read_csv(StringIO(data), sep=';')

    # Clean column names by stripping spaces
    population.columns = population.columns.str.strip()

    # Filter the columns to focus on 'OMRÅDE', 'TID', and 'INDHOLD' (which contains population)
    population = population[['OMRÅDE', 'TID', 'INDHOLD']]

    # Convert 'INDHOLD' to numeric
    population['INDHOLD'] = pd.to_numeric(population['INDHOLD'], errors='coerce')

    # Extract the year from 'TID' and group by area and year
    population['Year'] = population['TID'].str[:4]  # Extract year from 'TID'
    population_grouped = population.groupby(['OMRÅDE', 'Year'])['INDHOLD'].sum().reset_index()

    # Function to get population based on area and published year
    def get_population(location, published_date):
        year = published_date[:4]  # Extract the year from the published date
        matched_row = population_grouped[
            (population_grouped['OMRÅDE'] == location) &
            (population_grouped['Year'] == year)
        ]
        if not matched_row.empty:
            return matched_row.iloc[0]['INDHOLD']
        return None

    # Apply the function to add the 'Population' column
    df_16_skills['Population'] = df_16_skills.apply(
        lambda row: get_population(row['Area'], row['Published']), axis=1
    )

In [ ]:
# Looking at the first few rows of the dataset with population data included 
df_16_skills.info()

In [ ]:
# print the names of cites where the population is nan
df_16_skills[df_16_skills['Population'].isnull()]['Area'].unique()

In [ ]:
# Making a dictionary with population values for specific cities where the API did not return data
specific_population = {
    'Lyngby': 58538,
    'Taastrup': 57540,
    'Nørresundby': 24281,
    'Farum': 20426,
    'Birkerød': 20921,
    'Lillerød': 16801,
    'Frederiksværk': 12864,
}

# Fill missing population values (NaN) in the 'Population' column
df_16_skills['Population'] = df_16_skills.apply(
    lambda row: specific_population[row['Area']] if pd.isna(row['Population']) and row['Area'] in specific_population else row['Population'],
    axis=1
)

# Display the updated DataFrame
print(df_16_skills)

In [ ]:
# Looking into the final dataset for 2016
df_16_skills.info()

In [ ]:
# Turn df_16_skills into a csv file which will be used to create the final dataset. 
# The whole process is repeated for the other years as well.

df_16_skills.to_csv('updated_job_listings_with_skills_2016.csv', index=False)
df_17_skills.to_csv('updated_job_listings_with_skills_2017.csv', index=False)
df_18_skills.to_csv('updated_job_listings_with_skills_2018.csv', index=False)
df_19_skills.to_csv('updated_job_listings_with_skills_2019.csv', index=False)
df_20_skills.to_csv('updated_job_listings_with_skills_2020.csv', index=False)
df_21_skills.to_csv('updated_job_listings_with_skills_2021.csv', index=False)
df_22_skills.to_csv('updated_job_listings_with_skills_2022.csv', index=False)
df_23_skills.to_csv('updated_job_listings_with_skills_2023.csv', index=False)
df_24_skills.to_csv('updated_job_listings_with_skills_2024.csv', index=False)

In [ ]:
# Combine all dataframes into one
df = pd.concat([df_16_skills, df_17_skills, df_18_skills, df_19_skills, df_20_skills, df_21_skills, df_22_skills, df_23_skills, df_24_skills], ignore_index=True)

# Save the combined DataFrame to a CSV file which will be used for our streamlit app
df.to_csv('df_2016_2024.csv', index=False)
